 In this notebook only inference and submission task will be performed.

In [ ]:
import gc
import sys
import warnings
from pathlib import Path

import os

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
#warnings.simplefilter("ignore")

In [ ]:
# Helper function to unpack json found in daily data
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

# helper function to add is_played column
def is_played_games(row):
    if pd.isnull(row['gameDate']):
        is_played = 0
    else:
        is_played = 1
    return is_played

# helper function to add bmi column
def BMI(row):
    '''
    Calculate BMI for players.csv
    '''
    height_in = row['heightInches']
    mass_lb = row['weight']
    bmi = (mass_lb/height_in**2)*703
    
    return bmi

In [ ]:
# Helper function to unpack json found in daily data
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

# helper function to add is_played column
def is_played_games(row):
    if pd.isnull(row['gameDate']):
        is_played = 0
    else:
        is_played = 1
    return is_played

In [ ]:
# def age_now(row):
#     '''
#     Calculate Age (in years) at Given Day
#     [modified]
#     '''
#     date = row['gameDate']
#     given_day = pd.to_datetime(date)
#     dob = row["DOB"] #should be datetime formated already
#     age = (given_day - dob).days/365
    
#     return age

def age_now(row):
    '''
    Calculate Age (in days) at Given Day
    '''
    date = row['date']
    given_day = pd.to_datetime(date,format='%Y%m%d')
    dob = row["DOB"] #should be datetime formated already
    age = (given_day - dob).days/365
    
    return age

def age_now_d(row):
    '''
    Calculate Age (in days) from date_playerId
    '''
    date = row['date_playerId'].split('_')[0]
    given_day = pd.to_datetime(date,format='%Y%m%d')
    dob = row["DOB"] #should be datetime formated already
    age = (given_day - dob).days/365
    
    return age

def mlbDebutDays_now(row):
    '''
    Calculate mlbDebutDays at Given Day
    '''
    date = row['date']
    given_day = pd.to_datetime(date,format='%Y%m%d')
    dob = pd.to_datetime(row["mlbDebutDate"])
    mlbDebutDays = (given_day - dob).days
    
    return mlbDebutDays

In [ ]:
def evalYear(row):
    year = pd.to_datetime(row.date, format='%Y%m%d').year
    return year

In [ ]:
def player_data_process(dataset):
    '''
    This fucntion process the players.csv
    New Columns : age, bmi
    '''
    temp = dataset.copy()
    temp["DOB"] = pd.to_datetime(temp["DOB"]) # death of birth
    temp['bmi'] = temp.apply(BMI,axis=1)
    
    return temp

## Loading Data

In [ ]:
import os
os.getcwd()
os.listdir('../input/')

In [ ]:
test = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/example_test.csv")
players = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/players.csv")
mean_target_by_player = pd.read_csv("../input/derived-data/mean_target_by_player.csv")

## Loading Model

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import optimizers

In [ ]:
def make_model(n_in):
    inp = L.Input(name="inputs", shape=(n_in,))
    x = L.Dense(50, activation="relu", name="d3")(inp)
#     x = L.Dropout(0.2)(x)
    x = L.Dense(50, activation="relu", name="d4")(x)
#     x = L.Dropout(0.2)(x)
    preds = L.Dense(4, activation="linear", name="preds")(x)
    
    model = M.Model(inp, preds, name="ANN")
    model.compile(loss="mean_absolute_error", optimizer=optimizers.Adamax(lr=0.001, decay=1e-3))
    return model

In [ ]:
model = make_model(7)

In [ ]:
# Loads the weights
model.load_weights("../input/weights/model_ANN2.cpkt")

In [ ]:
print(model.summary())

## Prediction Task

In [ ]:
FECOLS = ['t1_m','t2_m','t3_m','t4_m','is_played','age','bmi'] #feature columns 
TGTCOLS = ['target1', 'target2', 'target3', 'target4']  #target columns

In [ ]:
def add_date_playerid(row):

    given_day = pd.to_datetime(row['date'],format='%Y%m%d') #taking timestamp of the given day
    next_day = given_day + pd.DateOffset(1) # next date
                                   
    next_day = str(next_day).split(" ")[0].replace("-","")
    playerId = row['playerId']
    date_playerId = next_day+"_"+str(playerId)

    return date_playerId

In [ ]:
#not used here, might be removed 
def process_prediction(test_df,sub_mode=True):
    append = False #flag for append to new_df
    
    for i in range(test_df.shape[0]):
        #test dataframe that is provided for submission has no formal date column
        if sub_mode:
            date = test_df.index[i]
        else:
            date = test_df.date.iloc[i] #taking the date where we are expanding json
        
        roster = unpack_json(test_df.rosters.iloc[i])
        roster.insert(0,'date',date) #inserting the given date
        
        if append==False:
            append= True
            new_df = roster
        else:
            new_df = new_df.append(roster,ignore_index=True)
            
    
    new_df['date_playerId'] = new_df.apply(add_date_playerid,axis=1)
    return new_df

In [ ]:
#processing players data
players_processed = player_data_process(players)

In [ ]:
# tempx = process_prediction(test,sub_mode=False)
# tempx

In [ ]:
# tempx[tempx.is_played==0]
# tempx[tempx.playerId==596049]

In [ ]:
# X = tempx[FECOLS].values
# preds = model.predict(X)

In [ ]:
# tempx[TGTCOLS] = np.clip(preds,0,100)
# tempx

In [ ]:
# gc.collect()

In [ ]:
# #to debug submission process
# sample_pred_temp = pd.DataFrame(columns=['date_playerId', 'target1', 'target2', 'target3', 'target4'])
# test_temp = pd.DataFrame(columns=['date_playerId', 'playerId', 'gameDate', 'teamId', 'statusCode',
#        'status', 'bmi', 'DOB', 'age', 'is_played', 'Unnamed: 0', 't1_m',
#        't2_m', 't3_m', 't4_m'])

## Submission

In [ ]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    sample_prediction_df.drop(TGTCOLS,axis=1,inplace=True)
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    
    
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
    
    #test_df extention here (added from rosters)
    test = sample_prediction_df.copy()
    test = test.merge(players_processed[['playerId','bmi','DOB']],on='playerId',how='left')
    #add new columns: age, is_played
    test['age'] = test.apply(age_now_d,axis=1)
    test = test.merge(test_rosters, on='playerId', how='left')
    test = test.merge(mean_target_by_player,how='inner', left_on=["playerId"],right_on=["playerId"])
    test['is_played'] = test.apply(is_played_games,axis=1)
    
    #making predictions : preds
    X = test[FECOLS].fillna(0.).values #extracting all features
    preds = model.predict(X) #model prediction
    
#     #to debug
#     sample_pred_temp = sample_pred_temp.append(sample_prediction_df)#,ignore_index=True)
#     test_temp = test_temp.append(test) #ignore_index=True)
#     #
    
    #merging prediction to submission dataframe
    sample_prediction_df[TGTCOLS] = np.clip(preds,0,100)
    sample_prediction_df = sample_prediction_df.fillna(0.)
    
    del sample_prediction_df['playerId']
   
    env.predict(sample_prediction_df)
    

In [ ]:
sample_prediction_df

In [ ]:
# sample_pred_temp

In [ ]:
# test_temp

In [ ]:
# # to check if any date_playerId is missing in submission
# list(test_temp.date_playerId)==list(sample_pred_temp.date_playerId)

In [ ]:
# # to reset env.predict() to work
# example_sample_submission = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv")
# example_sample_submission
# env.predict(example_sample_submission) 

In [ ]:
# # to check if any date_playerId is missing in submission
# list(example_sample_submission.date_playerId)==list(sample_pred_temp.date_playerId)